In [ ]:
!pip install datasets evaluate peft
!gdown 1WTjIveEsM7XpN28xm6F1qgX57QxaoFI_

!gdown 1WbyeG8f-V7VmpKdQam-0tNg4x6XDWoML

Downloading...
From: https://drive.google.com/uc?id=1WTjIveEsM7XpN28xm6F1qgX57QxaoFI_
To: /content/train_set.csv
100% 524k/524k [00:00<00:00, 121MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WbyeG8f-V7VmpKdQam-0tNg4x6XDWoML
To: /content/validation_set.csv
100% 112k/112k [00:00<00:00, 90.0MB/s]


In [ ]:
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, AutoPeftModelForSequenceClassification
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
train_df = pd.read_csv("train_set.csv")

In [ ]:
train_df.head()

,_id,label,title,label_numeric
0,66b5aabf8a38820e82e0b6ce,Xu hướng,"100+ STT Né thính, Cap né thính hài hước, NÉT ...",7
1,66b5a9838a38820e82e0b64d,Xu hướng,"Top 111+ stt cuộc sống an nhiên, bình dị tự tạ...",7
2,66b5cb358a38820e82e0c408,Xu hướng,"Top hạt giống hoa dễ trồng, nở quanh năm cho n...",7
3,66b5c7548a38820e82e0c271,Dinh dưỡng,Chi tiết 3 cách nấu rau bò khai đơn giản mà th...,1
4,66b5c7a78a38820e82e0c294,Nhà,Top 10 quạt cây hơi nước được ưa chuộng nhất h...,4


In [ ]:
train_df[["title", "label"]]

,title,label
0,"100+ STT Né thính, Cap né thính hài hước, NÉT ...",Xu hướng
1,"Top 111+ stt cuộc sống an nhiên, bình dị tự tạ...",Xu hướng
2,"Top hạt giống hoa dễ trồng, nở quanh năm cho n...",Xu hướng
3,Chi tiết 3 cách nấu rau bò khai đơn giản mà th...,Dinh dưỡng
4,Top 10 quạt cây hơi nước được ưa chuộng nhất h...,Nhà
...,...,...
4538,"Top 8 sữa tắm Hàn Quốc tốt nhất, được yêu thíc...",Làm Đẹp
4539,Ứng dụng đặt phòng khách sạn tốt nhất - Top ứn...,Xu hướng
4540,Review serum Teana Super Peptides Anti-redness...,Làm Đẹp
4541,Review Top 10 phấn bắt sáng đẹp nhất được ưa c...,Làm Đẹp


In [ ]:
labels = set(train_df["label"])
labels

{'Công Nghệ',
 'Dinh dưỡng',
 'Khuyến mãi',
 'Làm Đẹp',
 'Nhà',
 'Trẻ em',
 'Tài chính',
 'Xu hướng',
 'Đánh giá'}

In [ ]:
num_labels = len(labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
example = train_df["title"][0]
example

'100+ STT Né thính, Cap né thính hài hước, NÉT CĂNG, cực ngầu'

In [ ]:
tokenizer(example)

{'input_ids': [101, 2531, 1009, 2358, 2102, 11265, 4857, 2232, 1010, 6178, 11265, 4857, 2232, 15030, 15876, 10085, 1010, 5658, 2064, 2290, 1010, 12731, 2278, 12835, 4887, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
train_df = pd.read_csv("train_set.csv")
val_df = pd.read_csv("validation_set.csv")

In [ ]:
train_df = train_df.dropna(subset=['title', 'label'])
val_df = val_df.dropna(subset=['title', 'label'])

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['title'], padding=True, truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4543 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

In [ ]:
# Remove redundant columns
train_dataset = train_dataset.remove_columns(['_id', 'title', 'label'])  # Adjust based on what you want to keep
val_dataset = val_dataset.remove_columns(['_id', 'title', 'label'])  # Same for validation dataset


In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label_numeric'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label_numeric'])

In [ ]:
train_dataset = train_dataset.rename_column('label_numeric', 'labels')
val_dataset = val_dataset.rename_column('label_numeric', 'labels')

In [ ]:
train_args = TrainingArguments(
    output_dir="/results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [283]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [244]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=768, b

In [274]:
for param in base_model.parameters():
    param.requires_grad = False

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=[
        "q_lin", "k_lin", "v_lin", "out_lin",
        "lin1", "lin2"
    ]
)

peft_model = get_peft_model(model, lora_config)

In [281]:
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir='result_lora',
        learning_rate=5e-4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.05,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
    ),
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

In [ ]:
    trainer.evaluate()